In [2]:
from itertools import islice
import pbpstats
from pbpstats.client import Client
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score
from scipy.optimize import minimize_scalar

from math import exp, log
def sigmoid(x):
    return 1 / (1 + exp(-x))
sigmoid = np.frompyfunc(sigmoid, 1, 1)


def linear_loss(alpha, true_val, scaler):
    return sum(abs(true_val - sigmoid(alpha * scaler))) / len(true_val)

def quadratic_loss(alpha, true_val, scaler):
    return sum((true_val - sigmoid(alpha * scaler)) ** 2) / len(true_val)

def log_loss(alpha, true_val, scaler):
    return true_val*log(sigmoid(alpha * scaler)) + (1-true_val)*log(1 - sigmoid(alpha*scaler))
    

# def quadratic_loss_function(alpha, true_val, scaler):
#     return sum((true_val - sigmoid(alpha * scaler)) ** 2) / len(true_val)
    


In [1]:
from pbpstats.client import Client
season_settings = {
    "dir": "C:/Users/bhalb/nbaproject/response_data",
    "Games": {"source": "file", "data_provider": "stats_nba"},
    # "Boxscore": {"source": "web", "data_provider": "data_nba"},
    # "Possessions": {"source": "file", "data_provider": "data_nba"},
}
game_settings = {
    "dir": "C:/Users/bhalb/nbaproject/response_data",
    "Boxscore": {"source": "file", "data_provider": "stats_nba"},
}
game_possessions_settings = {
    "dir": "C:/Users/bhalb/nbaproject/response_data",
    # "Boxscore": {"source": "file", "data_provider": "stats_nba"},
    "Possessions": {"source": "file", "data_provider": "stats_nba"},
}
season_client = Client(season_settings)
game_client = Client(game_settings)
possession_client = Client(game_possessions_settings)

# season1718 = season_client.Season("nba", "2017-18", "Regular Season")
game = possession_client.Game("0020500717")


In [3]:
from nba_api.stats.static import players
# players.find_players_by_full_name("Smush Parker")
players.find_player_by_id("304")


{'id': 304,
 'full_name': 'John Stockton',
 'first_name': 'John',
 'last_name': 'Stockton',
 'is_active': False}

In [120]:
game.boxscore.team_items

[{'game_id': '0020500717',
  'team_id': 1610612753,
  'team_name': 'Magic',
  'team_abbreviation': 'ORL',
  'team_city': 'Orlando',
  'min': '290:00',
  'fgm': 32,
  'fga': 90,
  'fg_pct': 0.356,
  'fg3m': 7,
  'fg3a': 17,
  'fg3_pct': 0.412,
  'ftm': 18,
  'fta': 26,
  'ft_pct': 0.692,
  'oreb': 11,
  'dreb': 36,
  'reb': 47,
  'ast': 18,
  'stl': 6,
  'blk': 4,
  'to': 21,
  'pf': 28,
  'pts': 89,
  'plus_minus': -5.0},
 {'game_id': '0020500717',
  'team_id': 1610612749,
  'team_name': 'Bucks',
  'team_abbreviation': 'MIL',
  'team_city': 'Milwaukee',
  'min': '290:00',
  'fgm': 35,
  'fga': 88,
  'fg_pct': 0.398,
  'fg3m': 5,
  'fg3a': 15,
  'fg3_pct': 0.333,
  'ftm': 19,
  'fta': 26,
  'ft_pct': 0.731,
  'oreb': 13,
  'dreb': 39,
  'reb': 52,
  'ast': 21,
  'stl': 9,
  'blk': 7,
  'to': 18,
  'pf': 24,
  'pts': 94,
  'plus_minus': 5.0}]

In [109]:
dir(game.possessions.items[80].events[0])



['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_if_make_ends_possession_when_1_foul',
 '_check_if_make_ends_possession_when_not_1_foul',
 '_get_assisted_stat_items',
 '_get_blocked_stat_items',
 '_get_heave_stat_item',
 '_get_missed_stat_items',
 '_get_possessions_played_stats_items',
 '_get_seconds_played_stats_items',
 '_get_shot_distance_stat_items',
 '_get_unassisted_stat_items',
 '_is_jump_ball_possession_ending_event',
 'base_stats',
 'clock',
 'count_as_possession',
 'current_players',
 'data',
 'description',
 'distance',
 'event_action_type',
 'event_num',
 'event_stats',
 'event_type',
 'fouls_to_give',
 'game_id'

In [114]:
print(game.possessions.items[80].events[0].lineup_ids)
print(game.possessions.items[80].events[0].current_players)


{1610612753: '101143-1499-1883-2045-2730', 1610612749: '101106-2048-2072-2250-2551'}
{1610612753: [1883, 2730, 1499, 101143, 2045], 1610612749: [2250, 2048, 2551, 101106, 2072]}


In [115]:
from pbpstats.resources.enhanced_pbp import Foul, FreeThrow, Rebound, Substitution
schedule = season_client.Season("nba", "2017-18", "Regular Season")
game_id = schedule.games.final_games[6]['game_id']
game = possession_client.Game(game_id)

found = False
for pos in game.possessions.items:
    has_foul = False
    has_sub = False
    
    for event in pos.events:
        if isinstance(event, FreeThrow):
            has_foul = True
        elif has_foul and isinstance(event, Substitution):
            has_sub = True
        elif has_sub:
            if isinstance(event, Rebound) and event.is_real_rebound and event.oreb:
                print(pos.events)
                found = True
                break
    if found:
        break
        

[<StatsFoul GameId: 0021700007, Description: Dellavedova P.FOUL (P1.T1) (T.Washington), Time: 10:29, EventNum: 177>, <StatsFoul GameId: 0021700007, Description: Smart T.FOUL (P2.T2) (T.Washington), Time: 10:29, EventNum: 179>, <StatsFreeThrow GameId: 0021700007, Description: Dellavedova Free Throw Technical (6 PTS), Time: 10:29, EventNum: 180>, <StatsReplay GameId: 0021700007, Description: , Time: 10:29, EventNum: 205>, <StatsFieldGoal GameId: 0021700007, Description: MISS Smart 4' Layup, Time: 10:20, EventNum: 181>, <StatsRebound GameId: 0021700007, Description: Smart REBOUND (Off:1 Def:1), Time: 10:18, EventNum: 182>, <StatsFoul GameId: 0021700007, Description: Dellavedova S.FOUL (P2.T2) (M.Lindsay), Time: 10:18, EventNum: 183>, <StatsFreeThrow GameId: 0021700007, Description: MISS Smart Free Throw 1 of 2, Time: 10:18, EventNum: 185>, <StatsRebound GameId: 0021700007, Description: CELTICS Rebound, Time: 10:18, EventNum: 186>, <StatsSubstitution GameId: 0021700007, Description: SUB: B

In [117]:
dir(game.possessions.items[58].events[11])

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_get_ft_trip_stat_items',
 '_get_made_ft_stat_items',
 '_get_missed_ft_stat_items',
 '_get_possessions_played_stats_items',
 '_get_seconds_played_stats_items',
 '_is_jump_ball_possession_ending_event',
 'base_stats',
 'clock',
 'count_as_possession',
 'current_players',
 'data',
 'description',
 'event_action_type',
 'event_for_efficiency_stats',
 'event_num',
 'event_stats',
 'event_type',
 'foul_that_led_to_ft',
 'fouls_to_give',
 'free_throw_type',
 'game_id',
 'get_all_events_at_current_time',
 'get_offense_team_id',
 'is_away_from_play_ft',
 'is_end_ft',
 'is_first_ft',
 'is_fla

In [165]:
game.possessions.items[58].events[11].lineup_ids

{1610612749: '1626173-203089-203141-203507-203521',
 1610612738: '1626179-1627759-1628464-201143-203935'}

In [173]:
game.possessions.items[80].offense_team_id


1610612738

In [169]:
dir(game.possessions.items[40])


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'data',
 'end_time',
 'events',
 'game_id',
 'get_team_ids',
 'next_possession',
 'number',
 'offense_team_id',
 'period',
 'possession_has_timeout',
 'possession_start_type',
 'possession_stats',
 'previous_possession',
 'previous_possession_end_rebound_player_id',
 'previous_possession_end_shooter_player_id',
 'previous_possession_end_steal_player_id',
 'previous_possession_end_turnover_player_id',
 'previous_possession_ending_event',
 'previous_possession_has_timeout',
 'start_score_margin',
 'start_time']

In [78]:
game.possessions.items[40].previous_possession_end_steal_player_id


0

In [4]:

import importlib
import nba
importlib.reload(nba)
from nba import NbaTracker
from datetime import datetime

# data starts from 2000-01
seasons = [
    "2000-01",
    "2001-02",
    "2002-03",
    "2003-04",
    "2004-05",
    "2005-06",
    "2006-07",
    "2007-08",
    "2008-09",
    "2009-10",
    "2010-11",
    "2011-12",
    "2012-13",
    "2013-14",
    "2014-15",
    "2015-16",
    "2016-17",
    "2017-18",
    "2018-19",
    "2019-20",
    "2020-21"
] # 19-20 was bubble season
# season = season_client.Season("nba", "2020-21", "PlayIn")
# len(season.games.final_games)

# all_players = Players()
nbaTracker = NbaTracker()
# all_seasons = {}

# X, y, ey, sample_weights 
# intercept, players = nbaTracker.create_RAPM(seasons[10])
for season_name in seasons[18:]:
#     # schedule = season_client.Season("nba", season_name, "Regular Season")
    
    nbaTracker.load_season(season_name)
    print("Loaded ", season_name, datetime.now().time())
    # nbaTracker.create_RAPM(season_name)
    
    # season = Season(season_name, schedule)
    # all_seasons[season.name] = season
    # all_players.set_season(season)
    # all_teams.set_season(season)

    # for game_dict in schedule.games.final_games:
    #     game_id = game_dict['game_id']
    #     game = possession_client.Game(game_id)
    #     for team in game.boxscore.team_items:
    #         team_id = team['team_id']
    #         players = list(
    #             filter(lambda p: p['team_id'] == team_id, game.boxscore.player_items))
    #         nbaTracker.add_game(team, players)
    #     for possession in game.possessions.items:
    #         lineupCollecter.add_possession(possession)
        
    
# df = nbaTracker.build_win_prob_data()




Updating 2018-19
Loading possessions 11:37:02.643149
Loaded  2018-19 11:38:38.779213
Updating 2019-20
Loading possessions 11:38:41.222080
Loaded  2019-20 11:40:02.139801
Updating 2020-21
Loading possessions 11:40:07.915748
Loaded  2020-21 11:41:29.292396


In [7]:
for season_name in seasons[20:]:
    rapm_stuff = nbaTracker.create_RAPM(season_name)


loading RAPM
Top 20 RAPM
Nikola Jokic 0.028232434097304873 0.0720061176829516
Joe Ingles 0.0230203201944264 0.050633471070830736
Jayson Tatum 0.026201247664853322 0.049988693422162354
Jonas Valanciunas 0.016664642840801425 0.0462784641447161
Donte DiVincenzo 0.014763731935697645 0.045083734427945846
Kawhi Leonard 0.05379632894875814 0.044612678071380744
Stephen Curry 0.02934073475975081 0.04416012255570927
Enes Kanter 0.02966946238401659 0.04373821581636819
Joe Harris 0.03283795909073832 0.043698950392119316
Alex Caruso 0.027166322841000123 0.04127321486418007
Rudy Gobert 0.061033398673944056 0.041078075672087905
Zion Williamson 0.024181374161010297 0.040432549466292794
Jimmy Butler 0.030792222531723208 0.03901850811584662
LeBron James 0.04519760602714533 0.03853558390034467
CJ McCollum 0.02184310530839115 0.03732386791112727
Dorian Finney-Smith 0.030213072049987133 0.03579535285099165
Fred VanVleet 0.022401825811537615 0.03439946894342073
Kyrie Irving 0.015188930276398213 0.0343210456

In [8]:
rapm_stuff

({1629631: PlayerInfo(index=0),
  201568: PlayerInfo(index=1),
  1628381: PlayerInfo(index=2),
  1629629: PlayerInfo(index=3),
  1629027: PlayerInfo(index=4),
  203992: PlayerInfo(index=5),
  203524: PlayerInfo(index=6),
  1628989: PlayerInfo(index=7),
  1628981: PlayerInfo(index=8),
  1629164: PlayerInfo(index=9),
  1630233: PlayerInfo(index=10),
  1630219: PlayerInfo(index=11),
  203991: PlayerInfo(index=12),
  200765: PlayerInfo(index=13),
  203503: PlayerInfo(index=14),
  1630168: PlayerInfo(index=15),
  101150: PlayerInfo(index=16),
  1627739: PlayerInfo(index=17),
  1628369: PlayerInfo(index=18),
  202684: PlayerInfo(index=19),
  1628464: PlayerInfo(index=20),
  1627759: PlayerInfo(index=21),
  203935: PlayerInfo(index=22),
  201952: PlayerInfo(index=23),
  1629684: PlayerInfo(index=24),
  1629057: PlayerInfo(index=25),
  1630202: PlayerInfo(index=26),
  1628400: PlayerInfo(index=27),
  1629750: PlayerInfo(index=28),
  1630174: PlayerInfo(index=29),
  1629035: PlayerInfo(index=30

In [116]:
from nba_api.stats.static.players import find_player_by_id
# players.find_players_by_full_name("Smush Parker")
# player_data.find_player_by_id("304")

for pid, player in sorted(players.items(), key=lambda p: p[1].pm, reverse=True):
    name = find_player_by_id(pid)['full_name']
    print(name, player.pm)
    


Dirk Nowitzki 0.06053215774813888
John Stockton 0.05948640329326387
Vince Carter 0.05676071644828242
Shaquille O'Neal 0.04827192452514792
Karl Malone 0.04755730411158757
Shawn Marion 0.04639866907776238
Kobe Bryant 0.04499310500122287
Derek Fisher 0.04394205455216223
Steve Francis 0.04303724768993667
David Robinson 0.04295946356057785
Ray Allen 0.04189684111740386
Rashard Lewis 0.04178104078606208
Scott Williams 0.041219673458853856
Derek Anderson 0.040771951770568575
Tracy McGrady 0.03911488732660873
Mitch Richmond 0.038465958098156616
Paul Pierce 0.03820672958202611
Hakeem Olajuwon 0.035914116504534015
Eric Snow 0.03582840113002687
Aaron McKie 0.03479080287207919
Darrell Armstrong 0.034759370909138845
Vlade Divac 0.03423608636797793
Kurt Thomas 0.032902368454801545
Damon Stoudamire 0.0321769861184296
Arvydas Sabonis 0.03185670152470469
Jeff Foster 0.03173951781302225
Charlie Ward 0.031101017140623285
Lindsey Hunter 0.03105320607821385
Doug Christie 0.030577728811252543
Andre Miller 0